In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import re
import string
import pickle

In [22]:
df=pd.read_csv("mail_data.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [23]:
data=df.where((pd.notnull(df)), ' ')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
data.shape

(5572, 2)

In [25]:
#meta data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [26]:
data.loc[data.Category=='ham','Category']=1
data.loc[data.Category=='spam','Category']=0
data.isna().any()

data = data.drop_duplicates()

In [27]:
data.head()
data.duplicated().sum()

0

In [28]:
def preprocess(text):
    text = text.lower()
    text = re.sub('[.?]','',text)
    text = re.sub('\W'," ",text)
    text = re.sub("https?://\S+|www.\S+",'',text)
    text = re.sub('<.?>+','',text)
    text = re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text = re.sub('\n','',text)
    text = re.sub('\w\d\w','',text)
    return text

data['Message'] = data["Message"].apply(preprocess)

In [29]:
x=data.Message
y=data.Category

In [30]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=3)

In [31]:
f_ext=TfidfVectorizer(min_df=1,stop_words='english', )

x_train_features=f_ext.fit_transform(x_train)
x_test_features=f_ext.transform(x_test)

y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [32]:
model=LogisticRegression()
model.fit(x_train_features,y_train)

LogisticRegression()

In [33]:
prediction_training_data=model.predict(x_train_features)
accuracy=accuracy_score(y_train,prediction_training_data)
print("accuracy on training data",accuracy)

accuracy on training data 0.9621818181818181


In [34]:
prediction_testing_data=model.predict(x_test_features)
accuracy_test=accuracy_score(y_test,prediction_testing_data)
print("accuracy on training data",accuracy)

accuracy on training data 0.9621818181818181


In [35]:
input_your_mail = ["Dear Student, Please note, results of II, IV, and VI semester B. Tech, II Semester M.Tech/MCA MAKEUP EXAMINATIONS are updated on SLcM portal. Revaluation Notification for the makeup examination papers is uploaded on SLcM. -Academic Section"]

input_data_features = f_ext.transform(input_your_mail)
prediction = model.predict(input_data_features)
if(prediction[0]==1):
  print("Not a spam")
elif(prediction[0]==0):
  print("Spam Mail")

Not a spam


In [36]:
prediction

array([1])

In [37]:
pickle.dump(model,open("model.pkl","wb"))

In [38]:
pickle.dump(f_ext,open("f_ext.pkl","wb"))